In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
%%bash

# *Warning*: You need kaggle-api to be setup, for more information: https://github.com/Kaggle/kaggle-api

if [ ! -d data ]; then
    kaggle datasets download -d unsdsn/world-happiness
    mkdir -p data
    unzip world-happiness.zip -d data
fi